In [1]:
import numpy as np # importing numpy for matrix operations 
from scipy import *
from scipy.optimize import minimize
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython import display 
import itertools
from IPython.display import display
import random
import operator
from random import choices, randint, randrange, random, sample, seed, uniform, choice
from collections import namedtuple
from typing import List, Optional, Callable, Tuple
from functools import partial
sns.set()
import random
from itertools import product
import re
from collections import OrderedDict
from operator import getitem
from pprint import pprint
from numpy import asarray
import copy
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap
import sys

In [2]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

In [3]:
class IterRegistry(type):
    def __iter__(cls):
        return iter(cls._registry)

In [4]:
class MarketConfig: 
    #class attributes
    t0 = 2018 #The starting year for evaluation of the pay-offs
    T = 25  # Planning Horizon t = 2016,...,2040 # 100 year scope 
    year = np.arange(2018, 2018 + T)
    distance = np.array([23000]) #nautical miles #Doudnikoff & Lacoste(2014). 
    port_time =  np.array([240]) #hours ==> 10 days in a year  #Doudnikoff & Lacoste(2014).
    demand_18_21 = np.array([7000000,7200000,7200000,7800000]) #2018-2021 TEU data for europe asia based on unctad 2020in TEU 
    annual_working_time = np.array([6480])  #hours per year, assumption   #hours per year, assumption 
    initial_freight_rate = np.array([822])  ## in US$/TEU 
    #SFOC_main = np.array([206])  #g/kWh, specific daily main engine fuel oil consumption rate
    #SFOC_aux = np.array([221]) # specific fuel oil consumption of the auxiliary engine [g/kW h], 
    eng_load_main = np.array([0.8]) # % 
    eng_load_aux = np.array([0.5])  # engine load of the auxiliary engine [\%]


    def __init__(self, number_of_firms, pollution_decay_parameter, pollution_damage_parameter,
                 demand_income_elasticity, demand_price_elasticity, 
                 freight_rate, fuel_data, discount_rate ):
        '''__init__ a method to describe the poperty that the shipping market and simulation game has'''
        self.number_of_firms = number_of_firms
        self.pollution_decay_parameter = pollution_decay_parameter
        self.pollution_damage_parameter = pollution_damage_parameter
        self.demand_income_elasticity = demand_income_elasticity #Constant income elasticity #IMF
        self.demand_price_elasticity =  demand_price_elasticity #Constant own price elasticity#IMF
        self.freight_rate = freight_rate ## in US$/TEU
        self.fuel_data = fuel_data 
        self.discount_rate = discount_rate 
        #self.bau_industry_emission =  bau_industry_emission 
        #self.bau_pollution_stock = bau_pollution_stock
        #self.market_capacity = 0 #don 't need anymore
         
    
    #Don t need anymore
    #def get_market_capacity (self, firm_instance):
        #self.market_capacity += np.multiply(firm_instance.max_number_of_vessel, firm_instance.capacity) 
        #return self.market_capacity 
    
    def get_freight_rate_ratio(self):
        self.beta = np.true_divide(self.freight_rate, MarketConfig.initial_freight_rate) 
        #should be equal to 1, since we re keeping freight rate cst 
        return self.beta
   
    def get_market_demand(self):
        '''compute market level demand'''
        #---------> 1.Import real GDP growth data & compute GDP ratio : Source IMF@2020 #
        G_df = pd.read_csv('real_growth_rate.csv') #import IMF data
        
        #-------> 2.Construct GDP projection path 2022-2043 based on projection growth data 
        gdp_growth= G_df.values[:,7:28] #Read gdp growth from 2022-2,043; file begins with 2016 growth rates
        g =1 +(gdp_growth/100)  
        #self.gdp = np.array([100.00 for j in range(MarketConfig.T)])
        #for foo in range (1,MarketConfig.T):
        #    self.gdp[foo] = g[:,foo]* self.gdp[foo-1]
            
        #print(self.gdp)
        #--------->  "compute GDP ratio based on IMF@2020"
        gdp_ratio = np.array([np.float64(g[:,0]) for j in range(np.size(g))])
        for moo in range (np.size(g)):
            sub_g = g[:,:moo+1]
            #print(sub_g)
            gdp_ratio[moo] = np.prod(sub_g)  
    
        
        #--------->  "compute freight rate ratio"
        self.freight_rate_ratio_multiplied = np.power(self.beta, self.demand_price_elasticity)
        #print(freight_rate_ratio_multiplied)
        
        #---------> "Project Transport Demand (industry demand)"
        loo =np.multiply(np.power(gdp_ratio, self.demand_income_elasticity), self.freight_rate_ratio_multiplied ) #Will need to be updated in case fuel prices are varied over time  
        predict_demand= np.multiply(loo, MarketConfig.demand_18_21[3])
        self.planing_market_demand = np.concatenate((MarketConfig.demand_18_21, predict_demand), axis=None) #concatenate the predicted values with the Untac data        
        self.market_demand = self.planing_market_demand[:MarketConfig.T]
        #print(Y) # size = 1 dimesion array with size T=35 years
        #self.market_psy = np.multiply(np.power(gdp_ratio, self.demand_income_elasticity), MarketConfig.demand_18_21[3] )
        return self.market_demand #, self.gdp #, self.market_psy

In [5]:
class Firm:
    #class attributes intialisation, to be updated with the addition of each instance
    _registry = []
    
    def __init__(self, index, capacity, design_speed, 
                 min_speed, max_speed, main_engine_power, aux_engine_power,
                 fix_cost, fuel_type, SFOC_M, SFOC_A, market_share):
        '''__init__ a method to describe the poperty that all shipping firm have'''
        self._registry.append(self)
        self.index = index
        self.capacity = capacity # Vessel capacity in TEU per firm, 
        #self.max_number_of_vessel = max_number_of_vessel # # of vessels per firms
        self.design_speed = design_speed  #design speed of the vessel in knots
        self.min_speed = min_speed #min vessel speed
        self.max_speed = max_speed #max vessel speed         
        self.main_engine_power =  main_engine_power  # PS_m : main engine power in kW
        self.aux_engine_power =  aux_engine_power  # auxiliary engine power [kW] 
        self.fix_cost = fix_cost #Daily cost of vessel (USD/Day) $25,000
        self.fuel_type = fuel_type
        self.SFOC_M = SFOC_M
        self.SFOC_A = SFOC_A
        self.market_share = market_share # the firm's market share 
        #self.bau_emission = bau_emission
         
   
    # Firm methods # Reupdate to make market share as in input
    #def get_market_share(self , market_instance):
        #'''return market share of the firm accoding to it s capacity'''
        #self.transport_capacity = np.multiply(self.max_number_of_vessel , self.capacity)
        #self.market_share= np.true_divide(self.transport_capacity, market_instance.market_capacity)
        #return self.market_share
        
    #def get_firm_param(self, market_instance):
        #self.psy = np.multiply (self.market_share , market_instance.market_psy)
        #return self.psy
    
    def get_firm_demand(self, market_instance):
        self.firm_demand = np.multiply (self.market_share , market_instance.market_demand)
        #print("firme demand", self.firm_demand)
        return self.firm_demand 
    
    def update_vessel_speed(self, operational_speed):
        self.operational_speed = operational_speed
        #print(self.operational_speed)
        return self.operational_speed 
    
    def get_min_number_of_vessel(self, market_instance):
        self.time_at_sea = np.true_divide(market_instance.distance,  self.operational_speed) #returns a vector 
        #print( " operational_speed", self.operational_speed)
        #print( " self.time_at_sea", self.time_at_sea)
        self.voyage_time = self.time_at_sea + market_instance.port_time #returns a vector 
        self.number_of_trips_to_meet_demand = np.ceil(np.true_divide(self.firm_demand, self.capacity)) #returns a vector
        self.max_trips_per_vessel = np.floor(np.true_divide(market_instance.annual_working_time, self.voyage_time)) #returns a vector
        self.min_number_of_vessel_to_meet_demand = np.true_divide(self.number_of_trips_to_meet_demand, self.max_trips_per_vessel) #returns a vector
        #print("max_trips_per_vessel", self.number_of_trips_to_meet_demand)
        #print("min_number_of_vessel_to_meet_demand", self.min_number_of_vessel_to_meet_demand)
        return self.min_number_of_vessel_to_meet_demand, self.number_of_trips_to_meet_demand, self.time_at_sea 
    
    def get_number_of_vessel(self):
        self.number_of_vessel = np.ceil(self.min_number_of_vessel_to_meet_demand) #returns a vector
        #print("self.number_of_vessel", self.number_of_vessel)
        return self.number_of_vessel
            
    def get_ship_energy_efficiency(self, market_instance):
        '''ship effici = SFPC * ENGINE POWER * ENGINE LOAD)* (Vds)^-3 '''
        self.main_fuel_parameter = self.SFOC_M * market_instance.eng_load_main * self.main_engine_power * 10**(-6)#self.main_fuel_parameter =np.multiply(np.multiply(np.multiply(self.SFOC_M, market_instance.eng_load_main),self.main_engine_power), 10**(-6))
        self.ship_energy = np.multiply(self.main_fuel_parameter, np.power(self.design_speed, -3) )  
        #print("self.ship_energy", self.ship_energy)
        return self.ship_energy
        
    def get_main_fuel_cons(self, market_instance):
        ''' fuel consu = ship eff * d * V^2 * # of trips to meet demand'''
        self.moo_1 = np.multiply(market_instance.distance, self.ship_energy)
        self.moo_2 = np.power(self.operational_speed, 2)
        self.moo_3 = np.multiply(self.moo_1, self.moo_2)
        self.main_fuel_cons = np.multiply(self.moo_3 , self.number_of_trips_to_meet_demand)
        #print("self.main_fuel_cons ", self.main_fuel_cons )
        return self.main_fuel_cons #returns a vector
        
    def get_aux_fuel_cons(self, market_instance): 
        self.aux_fuel_parameter = np.multiply(np.multiply(np.multiply( self.SFOC_A, market_instance.eng_load_aux), self.aux_engine_power),  10**(-6)) #self.aux_fuel_parameter = self.SFOC_A * market_instance.eng_load_aux * self.aux_engine_power * 10**(-6)
        self.aux_fuel_cons =  np.true_divide((self.aux_fuel_parameter * self.number_of_trips_to_meet_demand * market_instance.distance), self.operational_speed)        
        #print("self.aux_fuel_cons",self.aux_fuel_cons)
        return self.aux_fuel_cons #returns a vector   
    
        
    def get_fuel_cost(self, market_instance):
        self.aux_fuel_cost =np.multiply(self.aux_fuel_cons, market_instance.fuel_data['MGO']['price'])  
        self.main_fuel_cost = np.multiply(self.main_fuel_cons, market_instance.fuel_data[self.fuel_type]['price'])   
        self.fuel_cost = self.main_fuel_cost + self.aux_fuel_cost 
        #print("self.fuel_cost", self.fuel_cost)
        return self.fuel_cost #returns a vector
    
    
    def get_firm_carbon_emission(self, market_instance):
        self.carbon_aux_emision_factor = market_instance.fuel_data['MGO']['carbon_factor']
        self.carbon_aux_emissions = np.multiply(self.carbon_aux_emision_factor, self.aux_fuel_cons )  
        self.carbon_main_emision_factor =  market_instance.fuel_data[self.fuel_type]['carbon_factor']
        self.carbon_main_emissions =np.multiply(self.carbon_main_emision_factor, self.main_fuel_cons)  
        self.carbon_emissions = self.carbon_main_emissions + self.carbon_aux_emissions 
        #print("self.carbon_emissions", self.carbon_emissions )
        return self.carbon_emissions
    
    def get_firm_sulfur_emission(self, market_instance):
        self.sulfur_aux_emision_factor = market_instance.fuel_data['MGO']['sulfur_factor']
        self.sulfur_aux_emissions = np.multiply(self.sulfur_aux_emision_factor, self.aux_fuel_cons)  
        self.sulfur_main_emision_factor =  market_instance.fuel_data[self.fuel_type]['sulfur_factor']
        self.sulfur_main_emissions = np.multiply(self.sulfur_main_emision_factor, self.main_fuel_cons)  
        self.sulfur_emissions = self.sulfur_main_emissions + self.sulfur_aux_emissions 
        #print("self.sulfur_emissions", self.sulfur_emissions)
        return self.sulfur_emissions      
    
    
    def get_firm_operating_cost(self):
        self.operating_cost = np.multiply(self.fix_cost, self.number_of_vessel)  #Fixed Cost
        return self.operating_cost
        
            
    def get_total_cost(self):
        self.total_cost = self.operating_cost + self.fuel_cost
        #print("self.total_cost", self.total_cost)
        return self.total_cost #returns a vector    
     
                 
    def get_revenue(self,market_instance):
        self.revenue = np.multiply(self.firm_demand, market_instance.freight_rate)
        #print("self.revenue", self.revenue)
        return self.revenue #returns a vector
    
    def get_period_profits(self):
        self.period_profit = self.revenue - self.total_cost
        #print("self.period_profit", self.period_profit)
        return self.period_profit #returns a vector

# 1. Simulation Parameters:

In [6]:
####################################### 1. Firm & Market specific Parameters ###########################################
                    #===========================>  Firm 1 Attributes <==========================#
index_1 = 1
capacity_1 = np.array([14000]) 
#max_number_of_vessel_1= np.array([100]) 
design_speed_1 = np.array([25.0]) #design speed of the vessel isn knots
min_speed_1 = 12.0
max_speed_1 = 28.0
main_engine_power_1 =np.array([89700])  # main engine power in kW
aux_engine_power_1 =np.array([14000])  # auxiliary engine power [kW] 
fix_cost_1 = np.array([50000*365])   #Daily cost of vessel (USD/Day) $25,000
fuel_type_1 = "HFO"
SFOC_M_1 = 175
SFOC_A_1 = 32
market_share_1 = 0.1 #UPDATE Firm 1's market share 
                    #===========================>  Firm 2 Attributes <==========================#
index_2 = 2
capacity_2 = np.array([12000])
#max_number_of_vessel_2 = np.array([100])
design_speed_2 = np.array([25.0])  #design speed of the vessel in knots
min_speed_2 = 12.0
max_speed_2 = 28.0
main_engine_power_2 = np.array([82100]) # main engine power in kW
aux_engine_power_2 =np.array([14000])   # auxiliary engine power [kW] 
fix_cost_2 =np.array([45862*365]) #Daily cost of vessel (USD/Day) $25,000
fuel_type_2 = "HFO"
SFOC_M_2 = 159
SFOC_A_2 = 28
market_share_2 = 0.1 #UPDATE Firm 2's market share 
                    #===========================>  Firm 3 Attributes <==========================#
index_3 = 3
capacity_3 = np.array([10000])
#max_number_of_vessel_3 = np.array([100])
design_speed_3 = np.array([25.0])  #design speed of the vessel in knots
min_speed_3 = 12.0
max_speed_3 = 28.0
main_engine_power_3 = np.array([74000]) # main engine power in kW
aux_engine_power_3 =np.array([12000])   # auxiliary engine power [kW] 
fix_cost_3 =np.array([41756*365]) #Daily cost of vessel (USD/Day) $25,000
fuel_type_3 = "HFO"
SFOC_M_3 = 143
SFOC_A_3 = 24
market_share_3 = 0.2 #UPDATE Firm 3's market share 
                    #===========================>  Firm 4 Attributes <==========================#
index_4 = 4
capacity_4 = np.array([8000])
#max_number_of_vessel_4 = np.array([100])
design_speed_4 = np.array([25.0])  #design speed of the vessel in knots
min_speed_4 = 12.0
max_speed_4 = 28.0
main_engine_power_4 = np.array([68500]) # main engine power in kW
aux_engine_power_4 =np.array([12000])   # auxiliary engine power [kW] 
fix_cost_4 =np.array([37618*365]) #Daily cost of vessel (USD/Day) $25,000
fuel_type_4 = "HFO"
SFOC_M_4 = 133
SFOC_A_4 = 24
market_share_4 = 0.1 #UPDATE Firm 3's market share 
                    #===========================>  Firm 5 Attributes <==========================#
index_5 = 5
capacity_5 = np.array([6000])
#max_number_of_vessel_5 = np.array([100])
design_speed_5 = np.array([25.0])  #design speed of the vessel in knots
min_speed_5 = 12.0
max_speed_5 = 28.0
main_engine_power_5 = np.array([57100]) # main engine power in kW
aux_engine_power_5 =np.array([12900])   # auxiliary engine power [kW] 
fix_cost_5 =np.array([33466*365]) #Daily cost of vessel (USD/Day) $25,000
fuel_type_5 = "HFO"
SFOC_M_5 = 114
SFOC_A_5 = 26
market_share_5 = 0.1 #UPDATE Firm 3's market share 


                    #===========================> Market Attributes <==========================#
number_of_firms = 5
pollution_decay_parameter =  np.array([1])
pollution_damage_parameter =  np.array([1.5])
year  = MarketConfig.year
income_elasticity = np.array([0.8]) #Constant income elasticity #IMF
price_elasticity = np.array([-0.7])   #Constant own price elasticity#IMF
freight_rate = np.array([822]) ## in US$/TEU
fuel_data = pd.DataFrame(np.array([[422.50, 525.50, 597.00], 
                                [3.114,3.206,3.206],
                                [0.07,0.01,0.002]]),
                         columns=['HFO', 'ULSFO', 'MGO'],
                         index = ['price', 'carbon_factor', 'sulfur_factor'])
discount = np.array([0.02])  ##3  to 5 % transport canada ,#to be updated using the Ramsey rule,#pure rate of time preference of 1.5% + growth rate of consumption g * rate of risk conversion , an elasticity value of 2

# 2. Set up firm and market objects 

In [7]:
""" This cell needs to run once throughout the kernel"""
####################################### 2.Simulation Game Initialisation ###########################################
firm_1 = Firm(index_1, capacity_1, design_speed_1, 
                 min_speed_1, max_speed_1, main_engine_power_1, aux_engine_power_1,
                 fix_cost_1, fuel_type_1, SFOC_M_1, SFOC_A_1, market_share_1) 

firm_2 = Firm(index_2, capacity_2, design_speed_2, 
                 min_speed_2, max_speed_2, main_engine_power_2, aux_engine_power_2,
                 fix_cost_2, fuel_type_2, SFOC_M_2, SFOC_A_2, market_share_2) 

firm_3 = Firm(index_3, capacity_3,  design_speed_3, 
                 min_speed_3, max_speed_3, main_engine_power_3, aux_engine_power_3,
                 fix_cost_3, fuel_type_3,SFOC_M_3, SFOC_A_3, market_share_3) 

firm_4 = Firm(index_4, capacity_4,  design_speed_4, 
                 min_speed_4, max_speed_4, main_engine_power_4, aux_engine_power_4,
                 fix_cost_4, fuel_type_4,SFOC_M_4, SFOC_A_4, market_share_4) 


firm_5 = Firm(index_5, capacity_5,  design_speed_5, 
                 min_speed_5, max_speed_5, main_engine_power_5, aux_engine_power_5,
                 fix_cost_5, fuel_type_5,SFOC_M_5, SFOC_A_5, market_share_5) 


game_config = MarketConfig(number_of_firms, pollution_decay_parameter, pollution_damage_parameter, 
                           income_elasticity, price_elasticity, freight_rate, 
                           fuel_data, discount)

####################################### 3.Get market capacity #################################
#for  firm_object in Firm._registry:
#    market_capacity_sim = game_config.get_market_capacity(firm_object)

# 3. Demand

In [8]:
## Get freight ratio for passethrough
beta_sim = game_config.get_freight_rate_ratio()

# Get market demand  
#market_demand_sim, gdp_sim, market_psy_sim = game_config.get_market_demand()
market_demand_sim = game_config.get_market_demand()

# 7.Get firm level demand  
firm_1_demand_sim = firm_1.get_firm_demand(game_config)
#print("firm 1 demand", firm_1_demand_sim)
    
# 9.Get ship energy efficiency  
firm_1_ship_energy_sim = firm_1.get_ship_energy_efficiency(game_config)

# 7.Get firm level demand  
firm_2_demand_sim = firm_2.get_firm_demand(game_config)
#print("firm 2 demand", firm_2_demand_sim)


# 9.Get ship energy efficiency  
firm_2_ship_energy_sim = firm_2.get_ship_energy_efficiency(game_config)
#print("firm 2 ship energy", firm_2_ship_energy_sim)

# 7.Get firm level demand  
firm_3_demand_sim = firm_3.get_firm_demand(game_config)
#print("firm 3 demand", firm_3_demand_sim)

# 9.Get ship energy efficiency  
firm_3_ship_energy_sim = firm_3.get_ship_energy_efficiency(game_config)


# 7.Get firm level demand  
firm_4_demand_sim = firm_4.get_firm_demand(game_config)
#print("firm 1 demand", firm_4_demand_sim)
    
# 9.Get ship energy efficiency  
firm_4_ship_energy_sim = firm_4.get_ship_energy_efficiency(game_config)


# 7.Get firm level demand  
firm_5_demand_sim = firm_5.get_firm_demand(game_config)
#print("firm 1 demand", firm_5_demand_sim)
    
# 9.Get ship energy efficiency  
firm_5_ship_energy_sim = firm_5.get_ship_energy_efficiency(game_config)


In [9]:
market_demand_sim

array([ 7000000.        ,  7200000.        ,  7200000.        ,
        7800000.        ,  8104290.37379516,  8336865.5491866 ,
        8562867.59239633,  8788190.97193528,  9019443.51303361,
        9256781.23570516,  9500364.26547593,  9750356.94141652,
       10006927.92701742, 10270250.32398178, 10540501.78901267,
       10817864.65367329, 11102526.04740139, 11394678.02376047,
       11694517.69001338, 12002247.34010533, 12318074.59114637,
       12642212.52348514, 12974879.82446861, 13316300.9359847 ,
       13666706.2058873 ])

 ## Vessel Speed Vectors

In [10]:
#====> Iinitialisation of chromo and population objects 
Chromo =  float
Population = List[Chromo]
#====> Iinitialisation of function objects; Define the Objects; They allow you to pass in only what you need during the evol function 
PopulateFunc = Callable[[], Population]
VesselProc = Callable[[Chromo, Firm], Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray , np.ndarray ,np.ndarray, np.ndarray, np.ndarray]]
Integrated_Solution_Combo_Func = Callable[[dict, list , np.ndarray, np.ndarray,] , dict]

SelectionFunc = Callable[[np.ndarray, np.ndarray], Tuple[Chromo, Chromo]]  #takes a population and a fitness fn to select 2 solutions to be the parents of our next generaation solution
CrossoverFunc = Callable[[Chromo, Chromo], Tuple[Chromo, Chromo]] #takes 2 genomes and returns 2 new genomes 
MutationFunc = Callable[[Chromo], Chromo] #takes 1 genome and sometimes returns a modified one
FitnessFunc = Callable[[Chromo], list] #int to refelct max # of vessel


### Compute CO2 reduction from the optimum wrs to the optimum 

### Compute The COST 

# 4. GA set up 

### 1) GA specific functions 

In [11]:
#======> Generate Chromo
def generate_chromo(firm: Firm) :
    '''generate the chromos, ie, each firm s vessel speed in the planing horizon based on speed constraint; [v1,..,vt]'''
    #while True:
    chromo = np.random.uniform(firm.min_speed, firm.max_speed)  
        #if (vessel_proc(chromo, firm)[0] != 0).all():
            #break
        #print("chromo", chromo)
    return chromo  

#======> Generate Population
def generate_population(firm:Firm, size: int ):
    '''generate the population of the firm s vessel speed based on our coev choice param population size'''
    return  np.array([generate_chromo(firm) for _ in range(size)])

#=====> Select Parent Chromo 
#def selection_pair (chromo, weights, size): 
#    '''randomly select 2 chromo from the population based on their weights, ie , solution fitness''' 
#    print(chromo)
#    print(weights)
#    weights = np.asarray(weights).astype('float64')
#    weights = weights / np.sum(weights)
#    selected_index = np.random.choice(np.arange(size),size = 2, replace=False,p= weights)
#    #print("chromo[selected_index[0]]", chromo[selected_index[0]])
#    #print("chromo[selected_index[1]", chromo[selected_index[1]])
#    return chromo[selected_index[0]],chromo[selected_index[1]]

def selection_pair (population: Population, fitness_func: FitnessFunc, weights: list):
    
    weights = np.asarray(weights).astype('float64')
    weights = weights / np.sum(weights)    
    return  np.random.choice(np.hstack( population),
                            size = 2, 
                            replace=False,
                            p= weights) 

#=======> Child Chromo based on parent chrossover
def BLX_alpha_crossover (a: Chromo, b: Chromo, firm, sim_game) :
    #child = np.zeros(sim_game.T)
    #lower = np.zeros(sim_game.T)
    #upper = np.zeros(sim_game.T)
    #for i in range(sim_game.T): 
    lower = (min(a,b)) - ( (0.5) * (max(a,b) - min (a,b)))
    #print("lower", lower)
    upper= (max(a,b)) + ( (0.5) * (max(a,b) - min (a,b)))
    #print("upper", upper)
    lower= firm.min_speed if (lower < firm.min_speed) else lower
    upper= firm.max_speed if (upper > firm.max_speed) else upper
    child = np.random.uniform (lower, upper, 2)
    #print("child", child)
    
    return child

#=====>#mutation prob = 0.5
def mutation(chromo: Chromo, firm: Firm, sim_game, variance: float, probability: float = 0.1 ) : 
    #mutated_chromo = np.zeros(sim_game.T)
    #print("chromo",chromo)
    #for i in range(sim_game.T): 
    mutated_chromo = np.random.normal(chromo, variance, 1) if random.random() > probability else chromo
    #print("mutated_chromo ",mutated_chromo[i])
    mutated_chromo = firm.min_speed  if mutated_chromo < firm.min_speed  else mutated_chromo
    mutated_chromo = firm.max_speed if mutated_chromo  > firm.max_speed else mutated_chromo
    #print("mutated_chromo", mutated_chromo)
    return mutated_chromo
#====> 
def fitness_similarity_chech(max_fitness, number_of_similarity):
    result = 0
    similarity = 0
    for n in range(len(max_fitness)-1):
        if np.round(max_fitness[n], 3 ) == np.round(max_fitness[n+1],3):
            similarity += 1
        else:
            similarity = 0
    if similarity == number_of_similarity-1:
        result = 1
    return result

### 2. Vessel Operations and Fitness Evaluation

In [12]:
def fitness (chromo: Chromo, firm: Firm, sim_game: MarketConfig): 
    
    operational_speed_ = np.repeat (firm.update_vessel_speed(chromo),sim_game.T) # firm.update_vessel_speed(chromo)
    #print("operational_speed", operational_speed_)
    min_number_vessels_sim , trips_to_meet_demand_sim, time_at_sea_sim  = firm.get_min_number_of_vessel(sim_game)
    #print(min_number_vessels_sim, trips_to_meet_demand_sim, time_at_sea_sim )
    number_vessels_sim  = firm.get_number_of_vessel()
    #print("number_vessels_sim", number_vessels_sim)
    
    #if (number_vessels_sim > firm.max_number_of_vessel).any():
        #true 
        #return np.zeros(9) #np.array([0,..,0])
    #else:
    main_fuel_con_sim = firm.get_main_fuel_cons(sim_game)
    #print("main_fuel_con_sim", main_fuel_con_sim)
    aux_fuel_con_sim = firm.get_aux_fuel_cons(sim_game)
    #print("aux_fuel_con_sim", aux_fuel_con_sim)
    fuel_cost_sim = firm.get_fuel_cost(sim_game) 
    #fixed cost
    fixed_cost_sim = firm.get_firm_operating_cost()
    
    #print("fuel_cost_sim", fuel_cost_sim)
    total_cost_sim = firm.get_total_cost()
    #print("total_cost_sim", total_cost_sim)
    revenue_sim = firm.get_revenue(sim_game)
    #print("revenue_sim", revenue_sim)
    profit_vector_sim = firm.get_period_profits()
    #print("profit_vector_sim", profit_vector_sim)
    
    # discount rate
    discount_multiplier = np.power( 1+ sim_game.discount_rate, - np.arange(1,MarketConfig.T + 1))
    
    # discounted_period_profit
    discounted_period_profit = np.multiply( discount_multiplier, profit_vector_sim) 
    #print("discounted_period_profit", discounted_period_profit)
    
    # NPV
    payoff= np.sum(discounted_period_profit) 
    #print("payoff", payoff)   
    
    
    
    carbon_emissions_sim = firm.get_firm_carbon_emission(sim_game) 
    #print("carbon_emissions_sim", carbon_emissions_sim)
    sulfur_emissions_sim = firm.get_firm_sulfur_emission(sim_game)

    
    return payoff, number_vessels_sim, main_fuel_con_sim, aux_fuel_con_sim, fuel_cost_sim, fixed_cost_sim, total_cost_sim , revenue_sim ,profit_vector_sim, carbon_emissions_sim, sulfur_emissions_sim

In [13]:
def run_evolution(populate_func: PopulateFunc,
                  fitness_func : FitnessFunc, 
                  mutation_func: MutationFunc,
                  crossover_func: CrossoverFunc,
                  firm,
                  size,
                  sim_game,
                  generation_limit,
                  number_of_similarity,
                  selection_func: SelectionFunc = selection_pair
                 ):
    """The evolutionnary main loop""" 
    #1. Genrate the 1st generation: Initial generation/population
    population =  populate_func(firm) 
    #print(population)
    
    #2 Simulate until you reach generation limit,
    i = 0
    progress = pd.DataFrame(columns=['Generation', 'Vessel Speed', 'Firm Net present value', 
                                'Number of vessels', 'main_fuel_con_sim', 'aux_fuel_con_sim',  'fuel_cost_sim',"fixed_cost_sim",
                                    'total_cost_sim','revenue_sim','profit_vector_sim', "carbon_emissions_sim",
                                    "sulfur_emissions_sim"])
    
    population_ledger =  pd.DataFrame(columns=['Generation', 
                                               'Vessel Speed'])
    
    # Create the next generation 
    for i in range(generation_limit):
        #Step 1 : Sort solutions in the population of generation i based on fitness to select the best 2 solutions;
        ## THE first 2 observations
        population = sorted(population, key=lambda chromo: fitness_func(chromo)[0], reverse=True) 
        ##payoff, number_vessels_sim, main_fuel_con_sim, aux_fuel_con_sim, fuel_cost_sim, total_cost_sim , revenue_sim ,profit_vector_sim
        ## remove emissions from BAU 
        progress = progress.append({'Generation': i,
                          'Vessel Speed': population[0],
                          'Firm Net present value':fitness_func(population[0])[0],
                          'Number of vessels': fitness_func(population[0])[1] , 
                          'main_fuel_con_sim': fitness_func(population[0])[2],
                          'aux_fuel_con_sim': fitness_func(population[0])[3], 
                          'fuel_cost_sim': fitness_func(population[0])[4], 
                          "fixed_cost_sim" : fitness_func(population[0])[5], 
                          'total_cost_sim' : fitness_func(population[0])[6], 
                          'revenue_sim' : fitness_func(population[0])[7],
                          'profit_vector_sim': fitness_func(population[0])[8],
                                   "carbon_emissions_sim":fitness_func(population[0])[9] , 
                                    "sulfur_emissions_sim" : fitness_func(population[0])[10]}, ignore_index=True) 
        
        population_ledger = population_ledger.append({'Generation': i,
                                                  'Vessel Speed': population}, ignore_index=True) 
        
        #Similarity check
        if fitness_similarity_chech(progress['Firm Net present value'], number_of_similarity) == 1:
            break
#         print(progress)
#         print (i)
#         print("Best vessel speed in the", i, "th generation is" ,population[0] ) 
#         print("Best fitness in the", i, "th generation is", fitness_func(population[0]))
    
              
        #Step 2: Implement elitism and pick the top 2 chromo(speed) in the population based on fitness for the next gernation 
        next_generation = population[0:2] #pick the top 2      
        max = np.sum([fitness_func(chromo)[0] for chromo in population])
        weights= np.hstack([fitness_func(chromo)[0] / max for chromo in population])
        weights[weights<0] = 0 #in case the NPV <0

        
        #Step 3: j in range 24 
        for j in range(int(len(population) / 2) - 1):
            #Step 1: Selection
            #parents = selection_func(population, weights = weights)
            parents = selection_func(population, fitness_func, weights = weights)

            #print("parent are", parents) 
            
            #Step 2: Crossover
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            

            #Step 3: mutation 
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            #print("offspring a ", offspring_a) 
            #print("offspring b", offspring_b) 
            
            #Step 4: Next generation 
            next_generation += [offspring_a, offspring_b]
            
        #print("next generation",i, "is", next_generation)       
        population = next_generation #update current population with our next generation and start into the next round of the algo by sorting the population and checking if we reached our fitness limit 
    
    #Sort the population one last time in case we run out of generation  
    population = sorted(population, key=lambda chromo: fitness_func(chromo)[0], reverse=True)
    progress = progress.append({'Generation': i,
                          'Vessel Speed': population[0],
                          'Firm Net present value':fitness_func(population[0])[0],
                          'Number of vessels': fitness_func(population[0])[1] , 
                          'main_fuel_con_sim': fitness_func(population[0])[2],
                          'aux_fuel_con_sim': fitness_func(population[0])[3], 
                          'fuel_cost_sim': fitness_func(population[0])[4], 
                         "fixed_cost_sim" : fitness_func(population[0])[5], 
                          'total_cost_sim' : fitness_func(population[0])[6], 
                          'revenue_sim' : fitness_func(population[0])[7],
                          'profit_vector_sim': fitness_func(population[0])[8],
                         "carbon_emissions_sim":fitness_func(population[0])[9] , 
                            "sulfur_emissions_sim" : fitness_func(population[0])[10]}, ignore_index=True) 
    population_ledger = population_ledger.append({'Generation': i,
                                                  'Vessel Speed': population}, ignore_index=True) 
    #print(progress)
    return progress, population_ledger


# Run Evolution 

In [14]:
## Update with the right firm before size 
results, my_population_ledger  = run_evolution(
    populate_func = partial(generate_population,
                            size=1000),
     fitness_func= partial(fitness,
                           firm = firm_3,
                           sim_game= game_config), 
    
    mutation_func = partial (mutation,
                             firm = firm_3,
                             sim_game= game_config,
                             variance = 4,
                             probability = 0.05),
    
    
    crossover_func = partial (BLX_alpha_crossover, 
                              firm = firm_3,
                              sim_game= game_config),
    
    firm = firm_3, 
    size=1000,
    sim_game = game_config,
    generation_limit = 500,
    number_of_similarity =50)


In [15]:
optimum_solution_period = results.iloc[[-1]]
optimum_solution_period

,Generation,Vessel Speed,Firm Net present value,Number of vessels,main_fuel_con_sim,aux_fuel_con_sim,fuel_cost_sim,fixed_cost_sim,total_cost_sim,revenue_sim,profit_vector_sim,carbon_emissions_sim,sulfur_emissions_sim
54,53,[16.666893119462703],1.074470e+10,"[35.0, 36.0, 36.0, 39.0, 41.0, 42.0, 43.0, 44....","[484621.737894202, 498468.07326260774, 498468....","[27820.42199925908, 28615.29119923791, 28615.2...","[221361476.193858, 227686089.7993968, 22768608...","[533432900.0, 548673840.0, 548673840.0, 594396...","[754794376.193858, 776359929.7993968, 77635992...","[1150800000.0, 1183680000.0, 1183680000.0, 128...","[396005623.806142, 407320070.20060325, 4073200...","[1598304.3647321695, 1643970.203724517, 164397...","[33979.16249659266, 34949.995710781026, 34949...."


In [16]:
carbon_optimum = np.sum(optimum_solution_period["carbon_emissions_sim"].values[0])
sulfur_optimum = np.sum(optimum_solution_period["sulfur_emissions_sim"].values[0])
NPV_optimum =  optimum_solution_period["Firm Net present value"].values.astype(np.float64)[0]

In [17]:
periodes = range(MarketConfig.T) #optimum_solution.keys()
periodes
metrics = ["Vessel Speed", "Net present value", "Number of vessels","main_fuel_con_sim","aux_fuel_con_sim","fuel_cost_sim","fixed_cost_sim","total_cost_sim","revenue_sim","profit_vector_sim","carbon_emissions_sim","sulfur_emissions_sim"]
BAU_output = pd.DataFrame(columns = metrics, index = periodes ) # 

In [18]:
for periode in periodes: 
        BAU_output["Vessel Speed"][periode] = optimum_solution_period["Vessel Speed"].values.astype(np.float64)[0]
        BAU_output["Net present value"][periode] = optimum_solution_period["Firm Net present value"].values.astype(np.float64)[0]
        BAU_output["Number of vessels"][periode] = optimum_solution_period["Number of vessels"].values[0][periode]
        BAU_output["main_fuel_con_sim"][periode] = optimum_solution_period["main_fuel_con_sim"].values[0][periode]
        BAU_output["aux_fuel_con_sim"][periode] = optimum_solution_period["aux_fuel_con_sim"].values[0][periode]
        BAU_output["fuel_cost_sim"][periode] = optimum_solution_period["fuel_cost_sim"].values[0][periode]
        BAU_output["fixed_cost_sim"][periode] = optimum_solution_period["fixed_cost_sim"].values[0][periode]# Add operating costs 
        BAU_output["total_cost_sim"][periode] = optimum_solution_period["total_cost_sim"].values[0][periode]
        BAU_output["revenue_sim"][periode] = optimum_solution_period["revenue_sim"].values[0][periode]
        BAU_output["profit_vector_sim"][periode] = optimum_solution_period["profit_vector_sim"].values[0][periode] #discounted ??
        BAU_output["carbon_emissions_sim"][periode] = optimum_solution_period["carbon_emissions_sim"].values[0][periode]
        BAU_output["sulfur_emissions_sim"][periode] = optimum_solution_period["sulfur_emissions_sim"].values[0][periode]


        
        

In [19]:
# discounted_period_profit
discount_multiplier = np.power( 1+ game_config.discount_rate, - np.arange(1,MarketConfig.T + 1))
BAU_output["DiscountedOperatingCosts"] = np.multiply( discount_multiplier, BAU_output["fixed_cost_sim"]) 
BAU_output["DiscountedFuelConsumptionCosts"] =np.multiply( discount_multiplier, BAU_output["fuel_cost_sim"]) 
BAU_output

,Vessel Speed,Net present value,Number of vessels,main_fuel_con_sim,aux_fuel_con_sim,fuel_cost_sim,fixed_cost_sim,total_cost_sim,revenue_sim,profit_vector_sim,carbon_emissions_sim,sulfur_emissions_sim,DiscountedOperatingCosts,DiscountedFuelConsumptionCosts
0,16.666893,10744700965.758642,35.0,484621.737894,27820.421999,221361476.193858,533432900.0,754794376.193858,1150800000.0,396005623.806142,1598304.364732,33979.162497,522973431.372549,217021055.092018
1,16.666893,10744700965.758642,36.0,498468.073263,28615.291199,227686089.799397,548673840.0,776359929.799397,1183680000.0,407320070.200603,1643970.203725,34949.995711,527368166.089965,218844761.437329
2,16.666893,10744700965.758642,36.0,498468.073263,28615.291199,227686089.799397,548673840.0,776359929.799397,1183680000.0,407320070.200603,1643970.203725,34949.995711,517027613.813692,214553687.683656
3,16.666893,10744700965.758642,39.0,540007.079368,30999.898799,246659930.616013,594396660.0,841056590.616013,1282320000.0,441263409.383987,1780967.720702,37862.495353,549130635.586437,227875648.683621
4,16.666893,10744700965.758642,41.0,564238.166263,32390.919899,257728004.425706,624878540.0,882606544.425706,1332345337.451925,449738793.026218,1860882.938938,39561.453478,565971746.079535,233432194.164343
5,16.666893,10744700965.758642,42.0,578084.501631,33185.789099,264052618.031245,640119480.0,904172098.031245,1370580696.286277,466408598.255032,1906548.777931,40532.286692,568407779.419906,234471168.203087
6,16.666893,10744700965.758642,43.0,595392.420841,34179.375599,271958385.038168,655360420.0,927318805.038168,1407735432.189956,480416627.151788,1963631.076671,41745.82821,570530684.29169,236756140.254379
7,16.666893,10744700965.758642,44.0,609238.75621,34974.244799,278282998.643707,670601360.0,948884358.643707,1444778595.78616,495894237.142453,2009296.915663,42716.661424,572351803.666993,237511859.808315
8,16.666893,10744700965.758642,46.0,626546.67542,35967.831299,286188765.650631,701083240.0,987272005.650631,1482796513.542725,495524507.892094,2066379.214404,43930.202942,586635092.885065,239469956.691761
9,16.666893,10744700965.758642,47.0,643854.594631,36961.417799,294094532.657554,716324180.0,1010418712.657554,1521814835.149929,511396122.492375,2123461.513144,45143.74446,587635323.222465,241259949.868203


In [20]:
BAU_output.to_csv("3_ouput_by_period.csv", index=False)

# DONE HERE WITH Computing optimum

# SPEED REDUCTION SIM

In [21]:
# Create a speed vector with 1% decrease from the optimum 

In [22]:
def fitness_for_speed (chromo: Chromo, firm: Firm, sim_game: MarketConfig): 
    
    operational_speed_ = np.repeat (firm.update_vessel_speed(chromo),sim_game.T) # firm.update_vessel_speed(chromo)
    #print("operational_speed", operational_speed_)
    min_number_vessels_sim , trips_to_meet_demand_sim, time_at_sea_sim  = firm.get_min_number_of_vessel(sim_game)
    #print(min_number_vessels_sim, trips_to_meet_demand_sim, time_at_sea_sim )
    number_vessels_sim  = firm.get_number_of_vessel()
    #print("number_vessels_sim", number_vessels_sim)
    
    #if (number_vessels_sim > firm.max_number_of_vessel).any():
        #true 
        #return np.zeros(9) #np.array([0,..,0])
    #else:
    main_fuel_con_sim = firm.get_main_fuel_cons(sim_game)
    #print("main_fuel_con_sim", main_fuel_con_sim)
    aux_fuel_con_sim = firm.get_aux_fuel_cons(sim_game)
    #print("aux_fuel_con_sim", aux_fuel_con_sim)
    fuel_cost_sim = firm.get_fuel_cost(sim_game) 
    #fixed cost
    fixed_cost_sim = firm.get_firm_operating_cost()
    
    #print("fuel_cost_sim", fuel_cost_sim)
    total_cost_sim = firm.get_total_cost()
    #print("total_cost_sim", total_cost_sim)
    revenue_sim = firm.get_revenue(sim_game)
    #print("revenue_sim", revenue_sim)
    profit_vector_sim = firm.get_period_profits()
    #print("profit_vector_sim", profit_vector_sim)
    
    # discount rate
    discount_multiplier = np.power( 1+ sim_game.discount_rate, - np.arange(1,MarketConfig.T + 1))
    
    # discounted_period_profit
    discounted_period_profit = np.multiply( discount_multiplier, profit_vector_sim) 
    #print("discounted_period_profit", discounted_period_profit)
    
    # NPV
    payoff= np.sum(discounted_period_profit) 
    #print("payoff", payoff)   
    
    
    
    carbon_emissions_sim = firm.get_firm_carbon_emission(sim_game) 
    #print("carbon_emissions_sim", carbon_emissions_sim)
    sulfur_emissions_sim = firm.get_firm_sulfur_emission(sim_game)
    
    total_carbon_emission = np.sum(carbon_emissions_sim)
    
    total_sulfur_emission = np.sum(sulfur_emissions_sim)


    
    #return payoff, number_vessels_sim, main_fuel_con_sim, aux_fuel_con_sim, fuel_cost_sim, fixed_cost_sim, total_cost_sim , revenue_sim ,profit_vector_sim, carbon_emissions_sim, sulfur_emissions_sim
    return payoff, total_carbon_emission, total_sulfur_emission, number_vessels_sim




sim = {'Percentage_Change_Vessel_Speed': np.arange(0.01, 1, 0.01), 
       'NewSpeed': np.arange(0.01, 1, 0.01) * optimum_solution_period["Vessel Speed"].values.astype(np.float64)[0]}

sim_df = pd.DataFrame(data=sim)
sim_df['Percentage_Reduction_Speed'] = 1- sim_df['Percentage_Change_Vessel_Speed']
sim_df = sim_df[sim_df['NewSpeed'].between(12, 28)]
sim_df

my_metrics = ["NewSpeed", "NewFleetSize","Net present value","carbon_emissions_sim","sulfur_emissions_sim"]
res =  pd.DataFrame(columns=my_metrics)
for i in sim_df['NewSpeed']:
    moo = fitness_for_speed (i, firm_3, game_config)
    res = res.append({"NewSpeed": i,
                      "NewFleetSize": moo[3] ,
                          "Net present value": moo[0],
                          "carbon_emissions_sim":moo[1],
                          "sulfur_emissions_sim": moo[2] }, ignore_index=True)
res
## merge the 2 dataset
result = pd.merge(res,sim_df)
result
carbon_optimum = np.sum(optimum_solution_period["carbon_emissions_sim"].values[0])
sulfur_optimum = np.sum(optimum_solution_period["sulfur_emissions_sim"].values[0])
NPV_optimum =  optimum_solution_period["Firm Net present value"].values.astype(np.float64)[0]


result["Change in Profits"] = (( res['Net present value']- NPV_optimum )/NPV_optimum)* 100

## Add CO2 Carbon reduction ; divide by the orginal 
result["Change in Carbon"] = (( res['carbon_emissions_sim'] - carbon_optimum)/carbon_optimum)*100
result["cost_effec_co2"] = (NPV_optimum - res['Net present value'])/(carbon_optimum -res['carbon_emissions_sim'] )


### sulfur 
result["Change in Sulfur"] = (( res['sulfur_emissions_sim'] -sulfur_optimum )/sulfur_optimum)*100
result["cost_effec_sox"] = (NPV_optimum - res['Net present value'])/(sulfur_optimum -res['sulfur_emissions_sim'] )



In [23]:
result

,NewSpeed,NewFleetSize,Net present value,carbon_emissions_sim,sulfur_emissions_sim,Percentage_Change_Vessel_Speed,Percentage_Reduction_Speed,Change in Profits,Change in Carbon,cost_effec_co2,Change in Sulfur,cost_effec_sox
0,12.000163,"[47.0, 48.0, 48.0, 52.0, 55.0, 56.0, 58.0, 59....",8.361271e+09,3.286924e+07,6.406707e+05,0.72,0.28,-22.182378,-43.302297,94.943965,-48.017458,4027.411002
1,12.166832,"[47.0, 48.0, 48.0, 52.0, 55.0, 56.0, 58.0, 59....",8.288200e+09,3.360138e+07,6.584739e+05,0.73,0.27,-22.862444,-42.039386,100.794420,-46.572947,4279.627220
2,12.333501,"[47.0, 48.0, 48.0, 52.0, 55.0, 56.0, 58.0, 59....",8.213765e+09,3.434614e+07,6.765242e+05,0.74,0.26,-23.555197,-40.754721,107.122082,-45.108386,4552.463367
3,12.500170,"[47.0, 48.0, 48.0, 52.0, 55.0, 56.0, 58.0, 59....",8.137978e+09,3.510344e+07,6.948216e+05,0.75,0.25,-24.260549,-39.448418,113.983300,-43.623776,4848.354583
4,12.666839,"[47.0, 48.0, 48.0, 52.0, 55.0, 56.0, 58.0, 59....",8.060845e+09,3.587322e+07,7.133661e+05,0.76,0.24,-24.978418,-38.120585,121.443858,-42.119122,5170.143750
5,12.833508,"[47.0, 48.0, 48.0, 52.0, 55.0, 56.0, 58.0, 59....",7.982375e+09,3.665542e+07,7.321575e+05,0.77,0.23,-25.708724,-36.771325,129.581030,-40.594425,5521.170014
6,13.000177,"[47.0, 48.0, 48.0, 52.0, 55.0, 56.0, 58.0, 59....",7.902578e+09,3.744999e+07,7.511960e+05,0.78,0.22,-26.451393,-35.400735,138.486182,-39.049690,5905.381455
7,13.166846,"[47.0, 48.0, 48.0, 52.0, 55.0, 56.0, 58.0, 59....",7.821460e+09,3.825687e+07,7.704814e+05,0.79,0.21,-27.206354,-34.008910,148.268131,-37.484918,6327.479958
8,13.333514,"[47.0, 48.0, 48.0, 52.0, 55.0, 56.0, 58.0, 59....",7.739028e+09,3.907601e+07,7.900137e+05,0.80,0.20,-27.973540,-32.595937,159.057497,-35.900113,6793.109554
9,13.500183,"[47.0, 48.0, 48.0, 52.0, 55.0, 56.0, 58.0, 59....",7.655289e+09,3.990736e+07,8.097928e+05,0.81,0.19,-28.752887,-31.161900,171.012445,-34.295276,7309.104246


# Output Results 

In [24]:
result.to_csv("Speed_Simulation_3.csv", index=False)

output_results = {
                  "OptimumSpeed" :optimum_solution_period["Vessel Speed"].values.astype(np.float64)[0] , 
                  "number_vessels_sim" : [optimum_solution_period["Number of vessels"].values[0]], 
                  "DiscountedFuelConsumption":np.sum(BAU_output["DiscountedFuelConsumptionCosts"]) ,
                  "DiscountedOperatingCost": np.sum(BAU_output["DiscountedOperatingCosts"]) , 
                  "NPV_optimum" :  optimum_solution_period["Firm Net present value"].values.astype(np.float64)[0] ,
                  "carbon_optimum": np.sum(optimum_solution_period["carbon_emissions_sim"].values[0]) ,
                  "sulfur_optimum" :np.sum(optimum_solution_period["sulfur_emissions_sim"].values[0]) }


output_results_df = pd.DataFrame(data=output_results, index=[0])
output_results_df.to_csv("Speed_optimisation_3.csv", index=False)

### carbon path for the coaltion 
emissions_results = {"carbon": optimum_solution_period["carbon_emissions_sim"].values[0],
                    "sulfur":  optimum_solution_period["sulfur_emissions_sim"].values[0]} 
emissions_results_df = pd.DataFrame(data=emissions_results)
emissions_results_df.to_csv("emissions_3.csv", index=False)


